LLaVA-OneVision
===

## LLaVA-OneVision: Easy Visual Task Transfer

Paper: https://arxiv.org/abs/2408.03326

![LLaVA-OneVision Overview](../assets/llava-onevision-overview.png)

```bash
pip install torch torchvision
pip install transformers
pip install matplotlib
pip install supervision
```